Set the storage account variable and import the needed libraries

In [5]:
import datetime
from pyspark.sql.functions import *

storage_account = 'adfcookbookv2fs@adfcookbookv2storage'

StatementMeta(adfcookbookv2, 5, 5, Finished, Available)

Loading ratings

In [6]:
ratings_df = spark.read.load(f'abfss://{storage_account}.dfs.core.windows.net/movielens/ratings.csv'
, format='csv'
## If header exists uncomment line below
, header=True
)

StatementMeta(adfcookbookv2, 5, 6, Finished, Available)

Adding date column

In [7]:
ratings_df = ratings_df.withColumn("date", from_unixtime(col("timestamp"),"yyyy-MM-dd")).drop(col("timestamp"))

StatementMeta(adfcookbookv2, 5, 7, Finished, Available)

Aggregating ratings

In [8]:
group_cols = ["movieid"]
agg_df = ratings_df.groupBy(group_cols) \
                    .agg(count("rating").alias("count_rating"), \
                        avg("rating").alias("avg_rating") \
                    )

StatementMeta(adfcookbookv2, 5, 8, Finished, Available)

Loading movies

In [9]:
movie_df = spark.read.load(f'abfss://{storage_account}.dfs.core.windows.net/movielens/movies.csv'
, format='csv'
## If header exists uncomment line below
, header=True
)

StatementMeta(adfcookbookv2, 5, 9, Finished, Available)

Joining movies and aggregated ratings

In [10]:
output_df = movie_df.join(agg_df, 'movieid', 'inner')

StatementMeta(adfcookbookv2, 5, 10, Finished, Available)

Writing the output to curated and enriched folders

In [12]:
ratings_df.write.mode("overwrite").parquet(f'abfss://{storage_account}.dfs.core.windows.net/curated/rating/')
output_df.write.mode("overwrite").parquet(f'abfss://{storage_account}.dfs.core.windows.net/enriched/movies_rating/')

StatementMeta(adfcookbookv2, 5, 12, Finished, Available)